In [29]:

import pandas as pd
import numpy as np


df = pd.read_csv("/rds/general/user/dla24/home/thesis/TGCA_dataset/df_merged_allslides.csv")

In [30]:
#Check and remove the missingness


missing_count = df.isnull().sum()

# Calculate % missing
missing_percent = (missing_count / len(df)) * 100

# Combine into a summary table
missing_summary = pd.DataFrame({
    "Missing Count": missing_count,
    "Missing %": missing_percent.round(1)
})

# Filter only columns with missing values
missing_summary = missing_summary[missing_summary["Missing Count"] > 0]

print(missing_summary)


df_clean = df.dropna()


                  Missing Count  Missing %
os_days                      47        9.1
tumor_stage                  88       17.0
tumor_grade                  98       18.9
age_at_diagnosis             35        6.7


In [31]:
df_clean.isnull().sum()# ccheck

slide_path          0
slide_id            0
patient_id          0
vital_status        0
os_days             0
tumor_stage         0
tumor_grade         0
age_at_diagnosis    0
dtype: int64

In [32]:
df_clean.head()

,slide_path,slide_id,patient_id,vital_status,os_days,tumor_stage,tumor_grade,age_at_diagnosis
1,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-B2-4099-01Z-00-DX1,TCGA-B2-4099,Alive,972.0,Stage I,G3,30355.0
2,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-B0-4696-01Z-00-DX1,TCGA-B0-4696,Dead,866.0,Stage III,G3,21317.0
4,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-AK-3428-01Z-00-DX1,TCGA-AK-3428,Alive,3728.0,Stage III,G2,22653.0
6,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-DV-5568-01Z-00-DX1,TCGA-DV-5568,Alive,370.0,Stage I,G2,9714.0
7,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-DV-A4VX-01Z-00-DX1,TCGA-DV-A4VX,Dead,1626.0,Stage IV,G4,21670.0


In [33]:
df_clean["event"] = df_clean["vital_status"].map({"Dead": 1, "Alive": 0}) #create event variable for survival analysis

/rds/general/ephemeral/user/dla24/ephemeral/ipykernel_3906844/2280438211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["event"] = df_clean["vital_status"].map({"Dead": 1, "Alive": 0}) #create event variable for survival analysis


In [34]:
print(df_clean.columns)

Index(['slide_path', 'slide_id', 'patient_id', 'vital_status', 'os_days',
       'tumor_stage', 'tumor_grade', 'age_at_diagnosis', 'event'],
      dtype='object')


In [35]:
df_clean["age_at_diagnosis"].describe() # age is in days so we convert them in years
df_clean["age_at_diagnosis_years"] = (df_clean["age_at_diagnosis"] / 365.25).round(1)

/rds/general/ephemeral/user/dla24/ephemeral/ipykernel_3906844/3962239849.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["age_at_diagnosis_years"] = (df_clean["age_at_diagnosis"] / 365.25).round(1)


In [36]:
df_clean.head()

df_clean = df_clean.drop(columns=["age_at_diagnosis"]) #drop this columns useless now

In [47]:
# Clean tumor_stage: extract Roman numeral and map to integer
stage_map = {"I": 1, "II": 2, "III": 3, "IV": 4}
df_clean["tumour_stage"] = (
    df_clean["tumor_stage"]
    .astype(str)
    .str.extract(r"(I{1,3}V?)")[0]     # extract I, II, III, IV
    .map(stage_map)
)

# Clean tumor_grade: extract number directly
df_clean["tumour_grade"] = (
    df_clean["tumor_grade"]
    .astype(str)
    .str.extract(r"([1-4])")[0]        # extract 1, 2, 3, 4
    .astype(float)
)

In [50]:
df_clean.head()

df_clean.isnull().sum()

slide_path                0
slide_id                  0
patient_id                0
vital_status              0
os_days                   0
tumor_stage               0
tumor_grade               0
event                     0
age_at_diagnosis_years    0
tumour_stage              0
tumour_grade              6
dtype: int64

In [51]:
df_clean.isnull().sum()
df_clean.dropna() # removing the tumor grade GX 

,slide_path,slide_id,patient_id,vital_status,os_days,tumor_stage,tumor_grade,event,age_at_diagnosis_years,tumour_stage,tumour_grade
1,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-B2-4099-01Z-00-DX1,TCGA-B2-4099,Alive,972.0,Stage I,G3,0,83.1,1,3.0
2,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-B0-4696-01Z-00-DX1,TCGA-B0-4696,Dead,866.0,Stage III,G3,1,58.4,3,3.0
4,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-AK-3428-01Z-00-DX1,TCGA-AK-3428,Alive,3728.0,Stage III,G2,0,62.0,3,2.0
6,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-DV-5568-01Z-00-DX1,TCGA-DV-5568,Alive,370.0,Stage I,G2,0,26.6,1,2.0
7,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-DV-A4VX-01Z-00-DX1,TCGA-DV-A4VX,Dead,1626.0,Stage IV,G4,1,59.3,4,4.0
...,...,...,...,...,...,...,...,...,...,...,...
513,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-BP-4967-01Z-00-DX1,TCGA-BP-4967,Alive,205.0,Stage III,G2,0,76.3,3,2.0
514,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-B2-5635-01Z-00-DX1,TCGA-B2-5635,Alive,755.0,Stage I,G2,0,74.4,1,2.0
515,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-AK-3445-01Z-00-DX1,TCGA-AK-3445,Alive,2392.0,Stage III,G3,0,69.8,3,3.0
517,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-CJ-4897-01Z-00-DX1,TCGA-CJ-4897,Alive,3341.0,Stage III,G3,0,79.7,3,3.0


In [52]:
df_clean = df_clean.drop(columns=["tumor_stage","tumor_grade"]) # drop original columns

In [53]:
df_clean.head()

,slide_path,slide_id,patient_id,vital_status,os_days,event,age_at_diagnosis_years,tumour_stage,tumour_grade
1,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-B2-4099-01Z-00-DX1,TCGA-B2-4099,Alive,972.0,0,83.1,1,3.0
2,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-B0-4696-01Z-00-DX1,TCGA-B0-4696,Dead,866.0,1,58.4,3,3.0
4,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-AK-3428-01Z-00-DX1,TCGA-AK-3428,Alive,3728.0,0,62.0,3,2.0
6,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-DV-5568-01Z-00-DX1,TCGA-DV-5568,Alive,370.0,0,26.6,1,2.0
7,/rds/general/user/dla24/home/thesis/TGCA_datas...,TCGA-DV-A4VX-01Z-00-DX1,TCGA-DV-A4VX,Dead,1626.0,1,59.3,4,4.0


In [55]:
# check the types 
df_clean.isnull().sum()
df_clean = df_clean.dropna(subset=["tumour_grade"])


In [60]:
df_clean.isnull().sum()
df_clean.head()
# check the types 
df_clean.info()

df_clean["tumour_grade"] = df_clean['tumour_grade'].astype(int)

<class 'pandas.core.frame.DataFrame'>
Index: 411 entries, 1 to 518
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   slide_path              411 non-null    object 
 1   slide_id                411 non-null    object 
 2   patient_id              411 non-null    object 
 3   vital_status            411 non-null    object 
 4   os_days                 411 non-null    float64
 5   event                   411 non-null    int64  
 6   age_at_diagnosis_years  411 non-null    float64
 7   tumour_stage            411 non-null    int64  
 8   tumour_grade            411 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 32.1+ KB


In [ ]:
df_clean["age_at_diagnosis_years"].describe() # checking for outliers

count    411.000000
mean      61.032117
std       12.086208
min       26.600000
25%       51.750000
50%       60.700000
75%       69.800000
max       90.000000
Name: age_at_diagnosis_years, dtype: float64

In [65]:
df_clean.to_csv("/rds/general/user/dla24/home/thesis/TGCA_dataset/df_clean.csv")

In [67]:

patients = df_clean['patient_id'].unique()
np.random.seed(42)
np.random.shuffle(patients)

n_train = int(0.7 * len(patients))
n_val = int(0.15 * len(patients))

train_pats = patients[:n_train]
val_pats = patients[n_train:n_train+n_val]
test_pats = patients[n_train+n_val:]

df_train = df_clean[df_clean['patient_id'].isin(train_pats)]
df_val = df_clean[df_clean['patient_id'].isin(val_pats)]
df_test = df_clean[df_clean['patient_id'].isin(test_pats)]

df_train.to_csv("/rds/general/user/dla24/home/thesis/TGCA_dataset/train_clean.csv", index=False)
df_val.to_csv("/rds/general/user/dla24/home/thesis/TGCA_dataset/val_clean.csv", index=False)
df_test.to_csv("/rds/general/user/dla24/home/thesis/TGCA_dataset/test_clean.csv", index=False)

In [71]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 411 entries, 1 to 518
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   slide_path              411 non-null    object 
 1   slide_id                411 non-null    object 
 2   patient_id              411 non-null    object 
 3   vital_status            411 non-null    object 
 4   os_days                 411 non-null    float64
 5   event                   411 non-null    int64  
 6   age_at_diagnosis_years  411 non-null    float64
 7   tumour_stage            411 non-null    int64  
 8   tumour_grade            411 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 32.1+ KB
